In [22]:
import requests
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import json
import pandas as pd
import sqlite3
import hashlib
import pickle
from sentence_transformers import SentenceTransformer

In [23]:
sitemap_root = "https://samakal.com/sitemap.xml"
needed_article = 100  # Start with 10 for testing

In [24]:
response = requests.get(sitemap_root)
if response.status_code == 200:
    sitemap_content = response.text
    root = ET.fromstring(sitemap_content)
    
    # Define namespace
    ns = {'sm': 'http://www.sitemaps.org/schemas/sitemap/0.9'}
    
    # Extract all sitemap locations
    sitemaps = [loc.text for loc in root.findall('.//sm:loc', ns)]
    
    print("Loaded sitemap index with the following sub-sitemaps:")
    for sitemap in sitemaps:
        print(sitemap)
else:
    print(f"Failed to load sitemap: {response.status_code}")

Loaded sitemap index with the following sub-sitemaps:
https://samakal.com/sitemap/sitemap-static-pages.xml
https://samakal.com/sitemap/sitemap-section.xml
https://samakal.com/sitemap/sitemap-daily-2025-12-25.xml
https://samakal.com/sitemap/sitemap-daily-2025-12-24.xml
https://samakal.com/sitemap/sitemap-daily-2025-12-23.xml
https://samakal.com/sitemap/sitemap-daily-2025-12-22.xml
https://samakal.com/sitemap/sitemap-daily-2025-12-21.xml
https://samakal.com/sitemap/sitemap-daily-2025-12-20.xml
https://samakal.com/sitemap/sitemap-daily-2025-12-19.xml
https://samakal.com/sitemap/sitemap-daily-2025-12-18.xml
https://samakal.com/sitemap/sitemap-daily-2025-12-17.xml
https://samakal.com/sitemap/sitemap-daily-2025-12-16.xml
https://samakal.com/sitemap/sitemap-daily-2025-12-15.xml
https://samakal.com/sitemap/sitemap-daily-2025-12-14.xml
https://samakal.com/sitemap/sitemap-daily-2025-12-13.xml
https://samakal.com/sitemap/sitemap-daily-2025-12-12.xml
https://samakal.com/sitemap/sitemap-daily-2025-

In [25]:
# collect wanted number of articles

if 'sitemaps' in locals() and sitemaps:
    article_urls = []
    
    # Filter for daily sitemaps
    daily_sitemaps = [s for s in sitemaps if 'daily' in s]
    daily_sitemaps.sort(reverse=True)  # Latest first
    
    for sub_sitemap_url in daily_sitemaps:
        if len(article_urls) >= needed_article:
            break
        print(f"Fetching sub-sitemap: {sub_sitemap_url}")
        
        response = requests.get(sub_sitemap_url)
        if response.status_code == 200:
            sub_content = response.text
            sub_root = ET.fromstring(sub_content)
            
            # Extract article URLs
            urls = [url.text for url in sub_root.findall('.//sm:loc', ns)][::-1]  # Reverse for latest
            article_urls.extend(urls)
            print(f"Added {len(urls)} URLs. Total so far: {len(article_urls)}")
        else:
            print(f"Failed to load {sub_sitemap_url}: {response.status_code}")
    
    # Trim
    if len(article_urls) > needed_article:
        article_urls = article_urls[:needed_article]
    
    print(f"Collected {len(article_urls)} latest article URLs.")
    for url in article_urls[:5]:
        print(url)
else:
    print("No sitemaps loaded yet. Run the previous cell first.")

Fetching sub-sitemap: https://samakal.com/sitemap/sitemap-daily-2025-12-25.xml
Added 142 URLs. Total so far: 142
Collected 100 latest article URLs.
https://samakal.com/bangladesh/article/330965/%E0%A6%AA%E0%A7%8D%E0%A6%B0%E0%A6%A7%E0%A6%BE%E0%A6%A8-%E0%A6%89%E0%A6%AA%E0%A6%A6%E0%A7%87%E0%A6%B7%E0%A7%8D%E0%A6%9F%E0%A6%BE%E0%A6%B0-%E0%A6%AC%E0%A6%BF%E0%A6%B6%E0%A7%87%E0%A6%B7-%E0%A6%B8%E0%A6%B9%E0%A6%95%E0%A6%BE%E0%A6%B0%E0%A7%80-%E0%A6%96%E0%A7%8B%E0%A6%A6%E0%A6%BE-%E0%A6%AC%E0%A6%95%E0%A6%B6-%E0%A6%9A%E0%A7%8C%E0%A6%A7%E0%A7%81%E0%A6%B0%E0%A7%80%E0%A6%B0-%E0%A6%AA%E0%A6%A6%E0%A6%A4%E0%A7%8D%E0%A6%AF%E0%A6%BE%E0%A6%97
https://samakal.com/politics/article/330966/%E0%A6%A6%E0%A7%87%E0%A6%B6%E0%A7%87%E0%A6%B0-%E0%A6%AA%E0%A6%A5%E0%A7%87-%E0%A6%A4%E0%A6%BE%E0%A6%B0%E0%A7%87%E0%A6%95-%E0%A6%B0%E0%A6%B9%E0%A6%AE%E0%A6%BE%E0%A6%A8
https://samakal.com/politics/article/330967/%E0%A6%A6%E0%A7%87%E0%A6%B6%E0%A7%87-%E0%A6%AB%E0%A6%BF%E0%A6%B0%E0%A7%87-%E0%A6%AA%E0%A7%8D%E0%A6%B0%E0%A6%A5%E0%A6%AE-%

In [26]:
# Bulk fetch and parse articles

articles_data = []
fetch_urls = article_urls[:5]

for url in fetch_urls:
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract from JSON-LD
        json_lds = soup.find_all('script', type='application/ld+json')
        data = None
        for json_ld in json_lds:
            try:
                d = json.loads(json_ld.string)
                if d.get('@type') == 'NewsArticle':
                    data = d
                    break
            except:
                pass
        
        if data:
            title = data.get('headline', '')
            pub_date = data.get('datePublished', '')
            
            # Extract content from div with class="dNewsDesc"
            content_div = soup.find('div', class_='dNewsDesc')
            content = content_div.get_text().strip() if content_div else ''
            
            articles_data.append({
                'title': title,
                'pub_date': pub_date,
                'link': url,
                'content': content
            })

print(f"Fetched {len(articles_data)} articles")

Fetched 5 articles


In [27]:
#save collected articles as csv
df = pd.DataFrame(articles_data)
df.to_csv('samakal_articles.csv', index=False)
print("Saved to samakal_articles.csv")

Saved to samakal_articles.csv


In [28]:
# Load the model
model_name = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
model = SentenceTransformer(model_name)

print(f"Loaded model: {model_name}")

Loaded model: sentence-transformers/paraphrase-multilingual-mpnet-base-v2


In [29]:
# Connect to DB and create table
conn = sqlite3.connect('articles.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS articles (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    hash TEXT UNIQUE,
    publisher TEXT,
    title TEXT,
    summary TEXT,
    link TEXT,
    published TEXT,
    language TEXT,
    embedding BLOB
)
''')

print("Table created with UNIQUE on hash")


Table created with UNIQUE on hash


In [30]:
# Process and insert data
for article in articles_data:
    title = article['title']
    pub_date = article['pub_date']
    content = article['content']
    link = article['link']
    
    # Generate hash
    hash_value = hashlib.sha256((title + pub_date).encode()).hexdigest()
    
    # Encode content
    embedding = model.encode(content)
    embedding_blob = pickle.dumps(embedding.tolist())
    
    # Insert
    cursor.execute('''
    INSERT OR IGNORE INTO articles (hash, publisher, title, summary, link, published, language, embedding)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', (hash_value, 'Samakal', title, content, link, pub_date, 'bangla', embedding_blob))

conn.commit()
conn.close()

print("Data inserted into articles.db")

OperationalError: database is locked

In [ ]:
# Check the table
conn = sqlite3.connect('articles.db')
df_db = pd.read_sql_query("SELECT id, hash, publisher, title, link, published, language FROM articles", conn)
conn.close()

print(f"Loaded {len(df_db)} rows")
df_db

Loaded 15 rows


,id,hash,publisher,title,link,published,language
0,1,1683043d93ea2aefa9f78794bfdf3a8cdea8a0bb09d938...,DhakaTribune,"Police arrest suspect with firearm, explosives...",https://www.dhakatribune.com/bangladesh/399501...,2025-12-24T22:58:00+06:00,English
1,2,87c31ebed9f581928e04c7943002c585c9177cce4f9cfb...,DhakaTribune,Registration deadline on ‘Postal Vote BD’ exte...,https://www.dhakatribune.com/bangladesh/electi...,2025-12-24T21:32:22+06:00,English
2,3,d053b31e868a111ab00014ce71b38216b54b1daaf0032d...,DhakaTribune,"'We will not step back from reform, even at th...",https://www.dhakatribune.com/bangladesh/politi...,2025-12-24T21:28:51+06:00,English
3,4,51941bdff89a863702128cae648cdec31c7d732fb653b3...,DhakaTribune,DMP issues detailed traffic guidelines ahead o...,https://www.dhakatribune.com/bangladesh/dhaka/...,2025-12-24T21:20:49+06:00,English
4,5,de314a1f3937a2c2949f0cafcbc28320a89c23b35b4634...,DhakaTribune,Youth killed as crude bomb thrown from flyover...,https://www.dhakatribune.com/bangladesh/399493...,2025-12-24T20:40:27+06:00,English
5,6,09819fe0cc49aa1915bea0bb48c691a8af5983978509e9...,DhakaTribune,Advisory Council approves Smoking and Tobacco ...,https://www.dhakatribune.com/bangladesh/399492...,2025-12-24T20:09:59+06:00,English
6,7,7d958e193f602555d19e3cb367b9e083c57e647e3fc2f5...,DhakaTribune,Kosovo: Will elections break the political gri...,https://www.dhakatribune.com/world/399491/koso...,2025-12-24T20:05:21+06:00,English
7,8,1637877f53428a410dcb469789591dfcd02628013ddb1e...,DhakaTribune,What interest do China and Russia have in Vene...,https://www.dhakatribune.com/world/europe/3994...,2025-12-24T19:55:55+06:00,English
8,9,1f977b245df045af304077e4be7654feee2586a26934fa...,DhakaTribune,"‘No internet shutdown, even for a minute’ as T...",https://www.dhakatribune.com/bangladesh/govern...,2025-12-24T19:53:55+06:00,English
9,10,f84c7a21e70b7067709d366b13f80c4118416aa55a54fe...,DhakaTribune,"At UN, Russia and China criticize US conduct t...",https://www.dhakatribune.com/world/europe/3994...,2025-12-24T19:42:49+06:00,English
